In [ ]:
!pip install -U bitsandbytes transformers peft huggingface_hub sentencepiece protobuf

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login
login(token=os.environ.get("HF_TOKEN"))

username = 'iamraymondlow'
pretrained_model_name = "iamraymondlow/llama-31-8b-instructiontuned-uncensored"
adapter_name = "iamraymondlow/llama-31-8b-generalhealthcare-contexttuning-lora"
repo_name = "llama-31-8b-generalhealthcare-contexttuning-merged"

# Load the pre-trained base model
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name, quantization_config=quantization_config, device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Load the adapter using PeftModel (replace with the correct adapter path)
adapter_model = PeftModel.from_pretrained(base_model, adapter_name)

# Merge the adapter into the pre-trained model
merged_model = adapter_model.merge_and_unload()

# Upload the merged model to the Hugging Face Hub
merged_model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)